In [1]:
import pandas as pd
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
from methods.baseline import Baseline
from methods.retrieval import Retrieval

Using TensorFlow backend.


In [3]:
retrieval = Retrieval()

path = 'data/processed/eclipse'
path_buckets = 'data/normalized/eclipse/eclipse.csv'
path_train = 'data/processed/eclipse/train.txt'
path_test = 'data/processed/eclipse/test.txt'

MAX_SEQUENCE_LENGTH_T = 20 # Title
MAX_SEQUENCE_LENGTH_D = 200 # Description
MAX_SEQUENCE_LENGTH_I = 1682

# Create the instance from baseline
retrieval.baseline = Baseline(path, 'eclipse.csv', MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)

df = pd.read_csv(path_buckets)

# Load bug ids
retrieval.load_bugs(path, path_train)
# Create the buckets
retrieval.create_bucket(df)
# Read and create the test queries duplicate
retrieval.create_queries(path_test)

  0%|          | 0/212512 [00:00<?, ?it/s]

Reading train data


  0%|          | 618/321483 [00:00<00:51, 6179.21it/s]

Creating the buckets...


100%|██████████| 39523/39523 [00:01<00:00, 34527.96it/s]
12798it [00:00, 65043.65it/s]

Creating the queries...


In [4]:
import keras
# Read the siamese model
name = 'baseline_1000epoch_10steps_1024batch(eclipse)'
retrieval.read_model(name, MAX_SEQUENCE_LENGTH_I, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)

Loaded model from disk


In [5]:
retrieval.train_vectorized, retrieval.test_vectorized = [], []
# Infer vector to all train
retrieval.read_train(path_train)

100%|██████████| 22682/22682 [03:58<00:00, 95.04it/s]


In [224]:
retrieval.train_vectorized = []
infer_vector_train(retrieval, retrieval.train, retrieval.train_vectorized)


  3%|▎         | 1449/56578 [00:15<09:46, 93.97it/s]


  5%|▌         | 2909/56578 [00:31<09:30, 94.03it/s]


  8%|▊         | 4369/56578 [00:46<09:21, 92.92it/s]


 10%|█         | 5829/56578 [01:02<09:01, 93.67it/s]


 13%|█▎        | 7289/56578 [01:17<08:49, 93.00it/s]


...

 91%|█████████▏| 51669/56578 [09:10<00:52, 94.11it/s]


 94%|█████████▍| 53099/56578 [09:26<00:37, 93.83it/s]


 96%|█████████▋| 54529/56578 [09:41<00:21, 93.85it/s]


 99%|█████████▉| 55959/56578 [09:56<00:06, 93.87it/s]


100%|██████████| 56578/56578 [10:03<00:00, 93.78it/s]

In [215]:
def infer_vector_train(retrieval, bugs, vectorized):
    bug_set = retrieval.baseline.get_bug_set()
    bug_unique = set()
    for row in tqdm(bugs):
        dup_a_id, dup_b_id = row
        bug_unique.add(dup_a_id)
        bug_unique.add(dup_b_id)
    for bug_id in tqdm(bug_unique):
        bug = bug_set[bug_id]
        bug_vector = retrieval.model.predict([[retrieval.get_info(bug)], [bug['title_word']], [bug['description_word']]])[0]
        vectorized.append({ 'vector' : bug_vector, 'bug_id' : bug_id })

In [193]:
def infer_vector_test(retrieval, bugs, vectorized):
    bug_set = retrieval.baseline.get_bug_set()
    for row in tqdm(bugs):
        for bug_id in row:
            dup_a_id, dup_b_id = row
            bug = bug_set[bug_id]
            bug_vector = retrieval.model.predict([[retrieval.get_info(bug)], [bug['title_word']], [bug['description_word']]])[0]
            vectorized.append({ 'vector' : bug_vector, 
                               'dup_a' : dup_a_id if bug_id == dup_a_id else dup_b_id,
                               'dup_b' : dup_a_id if bug_id == dup_b_id else dup_b_id })

In [194]:
retrieval.test_vectorized = []
# Infer vector to all test
infer_vector_test(retrieval, retrieval.test, retrieval.test_vectorized)


  4%|▍         | 740/17572 [00:15<06:00, 46.65it/s]


  8%|▊         | 1475/17572 [00:31<05:43, 46.93it/s]


 13%|█▎        | 2205/17572 [00:47<05:26, 47.02it/s]


 17%|█▋        | 2935/17572 [01:02<05:11, 46.98it/s]


 21%|██        | 3665/17572 [01:18<04:55, 47.00it/s]


...

 91%|█████████ | 15950/17572 [05:39<00:34, 47.09it/s]


 95%|█████████▍| 16665/17572 [05:54<00:19, 46.90it/s]


 99%|█████████▉| 17380/17572 [06:09<00:04, 46.93it/s]


100%|██████████| 17572/17572 [06:14<00:00, 46.98it/s]

35144

In [232]:
from annoy import AnnoyIndex
import numpy as np
# Indexing all train
X = np.array(retrieval.train_vectorized)
annoy = AnnoyIndex(X[0]['vector'].shape[0])  # Length of item vector that will be indexed

In [233]:
loop = tqdm(total=len(X))
for index, row in enumerate(X):
    vector = row['vector']
    annoy.add_item(index, vector)
    loop.update(1)
loop.close()
annoy.build(10) # 10 trees
# annoy.save('eclipse_annoy.ann')


100%|██████████| 56578/56578 [00:01<00:00, 29544.86it/s]

True

In [234]:
# Creating a hash from the train to index the clusters indices
clusters_by_issue = { 'train' : {} } # { index_cluster : issue_id } 

In [235]:
len(retrieval.train), len(retrieval.train_vectorized), len(retrieval.test), len(retrieval.test_vectorized)

(70287, 56578, 17572, 35144)

In [237]:
for index, row in tqdm(enumerate(retrieval.train_vectorized)):
    bug_id = row['bug_id']
    clusters_by_issue['train'][index] = bug_id


0it [00:00, ?it/s]
56578it [00:00, 1361710.75it/s]

In [238]:
# Recommend neighborhood instances from test sample
X_test = retrieval.test_vectorized
loop = tqdm(total=len(X_test))
distance_test, indices_test = [], []
for index, row in enumerate(X_test):
    vector = row['vector']
    rank, dist = annoy.get_nns_by_vector(vector, 30, include_distances=True)
    #rank = rank[::-1]
    #dist = dist[::-1]
    indices_test.append(rank)
    distance_test.append(dist)
    loop.update(1)
loop.close()


 80%|███████▉  | 28075/35144 [00:14<00:03, 1974.61it/s]


100%|██████████| 35144/35144 [00:17<00:00, 1976.89it/s]

In [239]:
issues_by_buckets = {}
for bucket in tqdm(retrieval.buckets):
    issues_by_buckets[bucket] = bucket
    for issue in np.array(retrieval.buckets[bucket]).tolist():
        issues_by_buckets[issue] = bucket


100%|██████████| 321483/321483 [00:00<00:00, 936106.27it/s]

In [240]:
# Generating the rank result
rank_queries = []
set_queries = {}
for index, row in tqdm(enumerate(X_test)):
    dup_a, dup_b = row['dup_a'], row['dup_b']
    rank_queries.append("{}:{}".format(dup_a, dup_b))


0it [00:00, ?it/s]
35144it [00:00, 797294.58it/s]

In [241]:
exported_rank = []
loop = tqdm(total=len(indices_test))
for query, rank, sim in zip(rank_queries, indices_test, distance_test):
    search, dup = query.split(":")
    rank = ["{}:{}:{}".format(
        issues_by_buckets[clusters_by_issue['train'][item]], clusters_by_issue['train'][item], similarity) 
            for item, similarity in zip(rank[:20], sim[:20]) if  
                #issues_by_buckets[clusters_by_issue['train'][item]] != int(dup) 
               clusters_by_issue['train'][item] != int(search) ] # 
    exported_rank.append("{}|{}".format(query, ",".join(rank)))
    loop.update(1)
loop.close()


100%|██████████| 35144/35144 [00:01<00:00, 25309.33it/s]

In [242]:
exported_rank[:20]

['98309:128463|121593:181845:0.0052652508020401,163100:163100:0.0055247885175049305,124133:153800:0.0063691153191030025,192814:192814:0.0071253469213843346,134593:132416:0.007310786284506321,192814:192203:0.00755036948248744,124133:124133:0.008183266967535019,338114:338116:0.00841464288532734,237222:254666:0.008423421531915665,163222:177169:0.009052067995071411,339607:339607:0.009235741570591927,166413:199519:0.009356851689517498,171445:120055:0.009362472221255302,171445:171445:0.00942909810692072,169570:200279:0.009594874456524849,172331:244711:0.00971356499940157,213562:245651:0.009798879735171795,42138:42138:0.009931867010891438,57082:59616:0.010221356526017189,308240:312487:0.010228041559457779',
 '128463:98309|183330:183330:0.026757797226309776,119518:119518:0.036399293690919876,240167:328677:0.036587949842214584,228154:228170:0.03791701793670654,168757:168757:0.03865160048007965,212493:215617:0.03943891450762749,284869:314774:0.040057189762592316,310525:315795:0.04224496334791183

### Visualizing the rank

In [19]:
def get_similar_bugs_rank(index):
    query, rank = exported_rank[index].split('|')
    similar_ids = []
    for row in rank.split(','):
        master_id, bug_id, sim = row.split(':')
        similar_ids.append(bug_id)
    df_query = df[df['bug_id'] == int(query.split(':')[0])]
    df_similar = df[df['bug_id'].isin(similar_ids)]
    return df_query, df_similar

In [20]:
def plot_rank(test_labels, tsne_features):
    obj_categories = ['anchor', 'positive', 'negative']
    groups = [0, 1, 2]
    colors = plt.cm.rainbow(np.linspace(0, 1, 3))
    plt.figure(figsize=(10, 10))

    for c_group, (c_color, c_label) in enumerate(zip(colors, obj_categories)):
        plt.scatter(tsne_features[np.where(test_labels == c_group), 0],
                    tsne_features[np.where(test_labels == c_group), 1],
                    marker='o',
                    color=c_color,
                    linewidth='1',
                    alpha=0.8,
                    label=c_label)
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.title('t-SNE on Testing Samples')
    plt.legend(loc='best')
    #plt.savefig('clothes-dist.png')
    plt.show(block=False)

def display_rank_at_position(index):
    query, rank = exported_rank[index].split('|')
    query_bug_id = int(query.split(':')[0])
    x_test_features = []
    x_test_features.append()
    tsne_features = Baseline.create_features(x_test_features)
    Baseline.plot_2d(valid_sim, tsne_features)

In [21]:
df_query, df_similar = get_similar_bugs_rank(2)

In [22]:
df_query

,bug_id,bug_severity,bug_status,component,creation_ts,delta_ts,description,dup_id,priority,product,resolution,short_desc,version
265192,294924,enhancement,CLOSED,JPA,2009-11-11 18:13:00 -0500,2009-11-17 11:15:51 -0500,"The CacheInterceptor annotation is supported, ...",239825,P3,EclipseLink,DUPLICATE,cache-interceptor needs in the eclipselink orm...,2.0


In [23]:
retrieval.buckets[14277]

{14277, 19242}

In [24]:
df_similar

,bug_id,bug_severity,bug_status,component,creation_ts,delta_ts,description,dup_id,priority,product,resolution,short_desc,version
24,25,normal,RESOLVED,Team,2001-10-10 21:35:00 -0400,2003-01-28 15:49:58 -0500,It would be helpful if there was a notion of d...,28126,P3,Platform,DUPLICATE,[CVS UI] Need VCM Prefs - Default Repo Connect...,2.0
6302,7522,normal,RESOLVED,Team,2002-01-13 18:35:00 -0500,2002-04-18 16:41:43 -0400,"When you tag a resource, the prompter should s...",14052,P3,Platform,DUPLICATE,"Tagging prompter should suggest, show existing",2.0
7334,8559,normal,RESOLVED,Team,2002-01-27 13:28:00 -0500,2002-02-08 14:46:08 -0500,Get should be enabled for outgoing changes whe...,[],P3,Platform,FIXED,"SyncView: ""Get"" should be enabled for outgoing...",2.0
11008,12254,normal,RESOLVED,Team,2002-03-25 18:05:00 -0500,2002-03-26 16:21:23 -0500,From \n\nhttp://dev.eclipse.org/bugs/show_bug....,11615,P3,Platform,DUPLICATE,"""Get"" on a branch silently does nothing",2.0
12788,14044,normal,RESOLVED,Team,2002-04-17 16:34:00 -0400,2002-04-26 10:05:31 -0400,"The ""Configure Tags"" menu in the repo view sho...",13094,P3,Platform,DUPLICATE,"""Configure Tags"" should be ""Discover Tags""?",2.0
12892,14149,normal,RESOLVED,Team,2002-04-18 14:52:00 -0400,2002-04-18 17:27:40 -0400,Build M5\n\nAfter catastrophically corrupting ...,[],P3,Platform,FIXED,Error updating marker state on checking projec...,2.0
12908,14167,normal,RESOLVED,Team,2002-04-18 16:58:00 -0400,2002-04-26 16:31:48 -0400,Build M5\n\nWhen you have a file which has no ...,[],P3,Platform,FIXED,VCM should detect that synchronize was ignored...,2.0
13409,14671,normal,RESOLVED,Team,2002-04-25 15:38:00 -0400,2002-04-25 17:37:13 -0400,You can easily lose focus on the widget you ar...,13094,P3,Platform,DUPLICATE,Configure Tags dialog loses widget focus frequ...,2.0
14737,16010,normal,RESOLVED,Team,2002-05-15 01:45:00 -0400,2002-05-16 14:41:00 -0400,"It would be handy if ""Create Patch"" was on the...",12843,P3,Platform,DUPLICATE,"Handy if ""Create Patch"" was in Sync view",2.0
17140,18429,normal,RESOLVED,SWT,2002-05-31 09:05:00 -0400,2002-07-12 15:11:53 -0400,I've tried the examples for the ScrolledCompos...,[],P3,Platform,FIXED,ScrolledComposite with LayoutManager is VERY s...,2.0


In [243]:
with open(os.path.join(path, 'exported_rank.txt'), 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [244]:
class Evaluation():
    def __init__(self):
        pass
    
    """
        Rank recall_rate_@k
        rank = "query:master|master:id:sim,master:id:sim"
    """
    def top_k_recall(self, rank, k):
        query, rank = rank.split('|')
        query_dup_id, query_master = query.split(":")
        query_master = int(query_master)
        hit = 0
        for pos, item in enumerate(rank.split(",")[:20]):
            if item.strip() == '': continue
            master, dup, sim = item.split(':')
            dup = int(dup)
            master = int(master)
            if dup == query_master and (pos+1) <= k:
                hit=1
                return [hit]
        return [hit]

    def evaluate(self, path):
        recall_at_5, recall_at_10, recall_at_15, recall_at_20 = [], [], [], []
        total = 0
        print("Evaluating...")
        with open(path, 'r') as file_input:
            for row in file_input:
                if row == '': continue
                recall_at_5 += self.top_k_recall(row, k=5)
                recall_at_10 += self.top_k_recall(row, k=10)
                recall_at_15 += self.top_k_recall(row, k=15)
                recall_at_20 += self.top_k_recall(row, k=20)
                total+=1
        
        report = {
            'recall_at_5' : round(sum(recall_at_5) / total, 4),
            'recall_at_10' : round(sum(recall_at_10) / total, 4),
            'recall_at_15' : round(sum(recall_at_15) / total, 4),
            'recall_at_20' : round(sum(recall_at_20) / total, 4)
        }

        return report

In [245]:
#from methods.evaluation import Evaluation
evaluation = Evaluation()
report = evaluation.evaluate(os.path.join(path, 'exported_rank.txt'))
report

Evaluating...


{'recall_at_10': 0.0275,
 'recall_at_15': 0.0319,
 'recall_at_20': 0.0341,
 'recall_at_5': 0.0215}